Ejemplo de patrón de filtrado usando el filtro de Bloom. Éste nos obliga a realizar esta operación en dos pasos: el primero es crear un fichero que almacenará el filtro de Bloom; lo haremos con un bitarray. El segundo paso será pasar por el Map todos los registros, y en base al archivo que contiene el filtro de Bloom seleccionará los posibles positivos o los negativos; aquí dependerá del objetivo de nuestro filtro. La salida del Map será la lista de los registros filtrados. 

In [ ]:
%%writefile construyeFiltro.py
#!/usr/bin/env python

import csv
import mmh3
from bitarray import bitarray

size =   5000000;
bit_array = bitarray(size)
bit_array.setall(0)

#Función para las operaciones hash usando mmh3

def mapf(url): 
    b1 = mmh3.hash(url, 41) % size
    bit_array[b1]=1

    b2 = mmh3.hash(url, 42) % size
    bit_array[b2]=1

    b3 = mmh3.hash(url, 43) % size
    bit_array[b3]=1

    b4 = mmh3.hash(url, 44) % size
    bit_array[b4]=1

    b5 = mmh3.hash(url, 45) % size
    bit_array[b5]=1

    b6 = mmh3.hash(url, 46) % size
    bit_array[b6]=1

    b7 = mmh3.hash(url, 47) % size
    bit_array[b7]=1

#======================================Programa principal======================================

r = csv.reader(open("archivos_datos/FBarchivoAprendizaje.csv")); #Fichero con el que vamos a crear el filtro
for row in r:
    url=row[1]
    mapf(url);
    

f=open("FBcreado.txt","wb") #Creación del fichero binario de filtro de Bloom(FB)
f.write(bit_array)
f.close()
print ("Fichero binario de filtro de Bloom creado correctamente.")

In [ ]:
!python construyeFiltro.py

In [ ]:
!pip install mmh3

In [5]:
%%writefile fbloom.py
#!/usr/bin/env python
from mrjob.job import MRJob
import csv
import mmh3
from bitarray import bitarray

size =  5000000;
ba=bitarray(size);
ba.setall(0)
f=open("FBcreado.txt","rb")
f.readinto(ba)
f.readlines

class fbloom(MRJob):
        
    def mapper(self,_, line):
        linea=line.split(",")
        #print("La URL es: ", linea[1])
        b1 = mmh3.hash(linea[1], 41) % size
        #print("El calculo de mmh3 para b1 es: ",b1)
        b2 = mmh3.hash(linea[1], 42) % size
        b3 = mmh3.hash(linea[1], 43) % size
        b4 = mmh3.hash(linea[1], 44) % size
        b5 = mmh3.hash(linea[1], 45) % size
        b6 = mmh3.hash(linea[1], 46) % size
        b7 = mmh3.hash(linea[1], 47) % size
        
        """Aplicamos la condición y lanzamos con yield los que la cumplan"""
        if ba[b1]==True and ba[b2]==True and ba[b3]==True and ba[b4]==True and ba[b5]==True and ba[b6]==True and ba[b7]==True:
            yield "Linea seleccionada: ",linea[1]
            
if __name__ == '__main__':
    fbloom.run()

Writing fbloom.py


In [6]:
!python fbloom.py archivos_datos/FBarchivoComprobacion.csv

Traceback (most recent call last):
  File "fbloom.py", line 4, in <module>
    import mmh3
ModuleNotFoundError: No module named 'mmh3'


In [ ]:
!python --version